<a href="https://colab.research.google.com/github/nguyenducminh2206/NLP-Projects/blob/main/NER_CMC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets torch

In [2]:
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import BertTokenizerFast
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification


In [4]:
train_data = pd.read_csv('/content/train_processed.csv')
val_data = pd.read_csv('/content/dev_processed.csv')

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-multilingual-cased')

In [6]:
unique_labels_train = set(label for labels in train_data['labels'].apply(eval) for label in labels)

In [7]:
unique_labels_val = set(label for labels in val_data['labels'].apply(eval) for label in labels)

In [8]:
all_unique_labels = unique_labels_train.union(unique_labels_val)

In [9]:
all_unique_labels

{'B-LOCATION',
 'B-MISCELLANEOUS',
 'B-ORGANIZATION',
 'B-PERSON',
 'I-LOCATION',
 'I-MISCELLANEOUS',
 'I-ORGANIZATION',
 'I-PERSON',
 'O'}

In [10]:
label_to_id = {label: idx for idx, label in enumerate(sorted(all_unique_labels))}
id_to_label = {id: label for label, id in label_to_id.items()}

In [11]:
label_to_id

{'B-LOCATION': 0,
 'B-MISCELLANEOUS': 1,
 'B-ORGANIZATION': 2,
 'B-PERSON': 3,
 'I-LOCATION': 4,
 'I-MISCELLANEOUS': 5,
 'I-ORGANIZATION': 6,
 'I-PERSON': 7,
 'O': 8}

In [12]:
def encode_examples(texts, token_labels, tokenizer, max_length=128):
    input_ids = []
    attention_masks = []
    labels = []

    for i, (text, label) in enumerate(zip(texts, token_labels)):
        # Tokenize input and get the corresponding labels
        tokenized_inputs = tokenizer(text, truncation=True, padding='max_length', max_length=max_length, is_split_into_words=True)
        word_ids = tokenized_inputs.word_ids(batch_index=0)  # Map tokens to their corresponding word ids.
        previous_word_idx = None
        label_ids = []

        # Align label ids with word ids
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)  # Special tokens
            elif word_idx != previous_word_idx:
                label_ids.append(label_to_id[label[word_idx]])
            else:
                label_ids.append(-100)  # Pad tokens
            previous_word_idx = word_idx

        input_ids.append(tokenized_inputs['input_ids'])
        attention_masks.append(tokenized_inputs['attention_mask'])
        labels.append(label_ids)

    return input_ids, attention_masks, labels

# Prepare the training and validation data
train_texts = train_data['tokens'].apply(eval).tolist()
train_labels = train_data['labels'].apply(eval).tolist()
val_texts = val_data['tokens'].apply(eval).tolist()
val_labels = val_data['labels'].apply(eval).tolist()

# Encode the training and validation datasets
train_input_ids, train_attention_masks, train_labels = encode_examples(train_texts, train_labels, tokenizer)
val_input_ids, val_attention_masks, val_labels = encode_examples(val_texts, val_labels, tokenizer)

In [13]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertForTokenClassification, Trainer, TrainingArguments

In [14]:
class NERDataset(Dataset):
  def __init__(self, input_ids, attention_masks, labels):
    self.input_ids = input_ids
    self.attention_masks = attention_masks
    self.labels = labels

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return {
        'input_ids': torch.tensor(self.input_ids[idx], dtype=torch.long),
        'attention_mask': torch.tensor(self.attention_masks[idx], dtype=torch.long),
        'labels': torch.tensor(self.labels[idx], dtype=torch.long)
    }

In [15]:
train_dataset = NERDataset(train_input_ids, train_attention_masks, train_labels)
val_dataset = NERDataset(val_input_ids, val_attention_masks, val_labels)

In [ ]:
model = BertForTokenClassification.from_pretrained('bert-base-multilingual-cased', num_labels=len(label_to_id))

In [17]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy='epoch',
    save_strategy='epoch',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer.train()